# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import googlemaps

# Import API key
from api_keys import g_key

### Use csv from Weather Py 

In [2]:
weather_file = pd.read_csv("../output_data/city.csv")
weather_file.dropna()
weather_file


,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Eureka,47,US,1619912976,73,40.8021,-124.1637,55.99,7.00
1,1,Taoudenni,22,ML,1619912977,30,22.6783,-3.9836,86.58,13.18
2,2,Yangi Marg`ilon,31,UZ,1619912613,82,40.4272,71.7189,60.80,6.91
3,3,George,0,ZA,1619912977,93,-33.9630,22.4617,48.20,4.61
4,4,Ushuaia,40,AR,1619912550,70,-54.8000,-68.3000,39.20,4.61
...,...,...,...,...,...,...,...,...,...,...
565,565,Katubao,86,PH,1619913127,68,5.8892,124.8294,84.88,2.62
566,566,Hwange,85,ZW,1619913127,65,-18.3693,26.5019,68.70,9.98
567,567,Severo-Yeniseyskiy,100,RU,1619913127,96,60.3725,93.0408,30.56,8.70
568,568,Umluj,0,SA,1619912779,57,25.0213,37.2685,80.82,8.52


### Humidity Heatmap


In [3]:
#Load gmaps gkey
gmaps.configure(api_key=g_key)

In [4]:

locations = weather_file[["Lat", "Lng"]]
humidity = weather_file["Humidity"].astype(float)

In [5]:
hfigure = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
hfigure.add_layer(heat_layer)
#Display figure
hfigure

Figure(layout=FigureLayout(height='420px'))

### Cities meeting ideal weather conditions

In [6]:
temp_file=weather_file.loc[(weather_file['Max Temp']>=60)&(weather_file['Max Temp']<=90)]
wind_file=temp_file.loc[(temp_file['Wind Speed']<=10)]
ideal_weather=wind_file.loc[(wind_file['Cloudiness']<=10)]
ideal_weather



,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
13,13,Atuona,2,PF,1619912916,68,-9.8000,-139.0333,79.39,6.82
17,17,Kiryat Gat,1,IL,1619912980,65,31.6100,34.7642,71.01,3.49
29,29,Port Alfred,0,ZA,1619912539,65,-33.5906,26.8910,60.73,5.79
31,31,Basti,9,IN,1619912984,42,26.8000,82.7167,80.44,9.86
34,34,Havelock,1,US,1619912985,55,34.8791,-76.9013,64.00,3.00
37,37,Wuning,8,CN,1619912986,80,29.2589,120.2311,67.50,3.94
46,46,Avera,1,US,1619912644,38,33.1940,-82.5271,77.00,3.24
47,47,Mar del Plata,0,AR,1619912700,82,-38.0023,-57.5575,61.00,3.44
54,54,Newala,10,TZ,1619912989,97,-10.9431,39.2787,65.19,2.24
63,63,Alyangula,4,AU,1619912991,78,-13.8483,136.4192,82.40,8.05


### Hotel Map

In [7]:
possible_hotels = ideal_weather.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
possible_hotels["Hotel Name"] = ""
possible_hotels

,City,Country,Lat,Lng,Hotel Name
13,Atuona,PF,-9.8000,-139.0333,
17,Kiryat Gat,IL,31.6100,34.7642,
29,Port Alfred,ZA,-33.5906,26.8910,
31,Basti,IN,26.8000,82.7167,
34,Havelock,US,34.8791,-76.9013,
37,Wuning,CN,29.2589,120.2311,
46,Avera,US,33.1940,-82.5271,
47,Mar del Plata,AR,-38.0023,-57.5575,
54,Newala,TZ,-10.9431,39.2787,
63,Alyangula,AU,-13.8483,136.4192,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [9]:
for index, row in possible_hotels.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    #base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    json.dumps(hotel_name, indent=4, sort_keys=True)
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        possible_hotels.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in possible_hotels.iterrows()]
locations = possible_hotels[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

hfigure.add_layer(markers)

# Display Map
hfigure


Figure(layout=FigureLayout(height='420px'))